In [35]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import numpy as np
from keras.preprocessing import image
import pandas as pd
import tensorflow as tf

# Loading the train and test value 
train = pd.read_csv('D:\Studies\Machine Learning\HackerEarth\PredictName\dataset\Train.csv',dtype=str)
test = pd.read_csv('D:\Studies\Machine Learning\HackerEarth\PredictName\dataset\Test.csv',dtype=str)



In [2]:
train.head(2)

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam


In [14]:
img_width,img_height = 150,150

train_data_dir = "D:\Studies\Machine Learning\HackerEarth\PredictName\dataset\train"
validation_data_dir = "D:\Studies\Machine Learning\HackerEarth\PredictName\dataset\test"

nb_train_samples = 273
nb_validation_samples = 91
epochs = 250
batch_size = 200

if K.image_data_format() == 'Channels_first':
    input_shape = (3, img_width,img_height)
else:
    input_shape = (img_width,img_height,3)


In [12]:
datagen=ImageDataGenerator(rescale=1. / 255,shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="D:/Studies/Machine Learning/HackerEarth/PredictName/dataset/train/",
x_col="Image",
y_col="target",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(150,150)
)

validation_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="D:/Studies/Machine Learning/HackerEarth/PredictName/dataset/train/",
x_col="Image",
y_col="target",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(150,150)
)
print(train_generator.class_indices)

Found 273 validated image filenames belonging to 8 classes.
Found 91 validated image filenames belonging to 8 classes.
{'bharatanatyam': 0, 'kathak': 1, 'kathakali': 2, 'kuchipudi': 3, 'manipuri': 4, 'mohiniyattam': 5, 'odissi': 6, 'sattriya': 7}


In [ ]:

##############################################################################################


model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.summary()

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

'''
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))



'''


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.fit_generator(train_generator,steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples//batch_size)

model.save_weights('first_tried.h5')

##############################################################################################


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_77 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_77 (Activation)   (None, 148, 148, 32)      0         
________________________________

In [6]:
import cv2
def load_image(path):    
    img_pred = image.load_img(path,target_size=(150, 150))
    img_pred = image.img_to_array(img_pred)
    img_pred = np.expand_dims(img_pred, axis = 0 )
    return img_pred

In [25]:
lst = []
for filename in test.Image:
    #print('{} is loaded'.format(filename))
    name = ('D:/Studies/Machine Learning/HackerEarth/PredictName/dataset/test/{}'.format(filename))
    img = load_image(name)
    result = model.predict_classes(img)
    lst.append(max(result)) 
d = { 0:'bharatanatyam', 1:'kathak', 2:'kathakali', 3:'kuchipudi', 4:'manipuri', 5:'mohiniyattam', 6:'odissi', 7:'sattriya'}
prediction = [d.get(key,0) for key in lst]  
    

In [26]:
submission = pd.DataFrame(test.Image)
submission['target']=prediction
submission.to_csv('submission.csv',index=False)